In [1]:
from platform import python_version
import torch

print('python ver:', python_version(),'torch ver:', torch.__version__)
!which nvcc

python ver: 3.9.24 torch ver: 2.8.0+cu128
/usr/local/cuda-12.8/bin/nvcc


# PointFlow

In [2]:
from pathlib import Path
cwd = Path.cwd()
module_dir = cwd / 'pytorch_structural_losses'

# pip uninstall PyTorchStructuralLosses
!cd {module_dir} && pip install .

Processing /mnt/c/Users/chawa/OneDrive/CMU_PYTHON/pytorch_pcd_metrics/pytorch_structural_losses
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'PyTorchStructuralLosses' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'PyTorchStructuralLosses'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for PyTorchStructuralLosses: filename=pytorchstructurallosses-0.1.0-cp39-cp39-linux_x86_64.whl size=4307579 sha256=0ec53e49eaffbb2e726e10eb3e13e81c6ebaff2128d68a61f138b02e8e02765d
  Stored in directory: /tmp/pip-ephem-wheel-cache-npglffbq/wheels/16/34/c5/da4241c0228918ee62a5864814267fd5ab0a3d4b1d2ae11217
Successfully built PyTorchStructuralLosses
  Attemp

In [3]:
import StructuralLosses

a, b = torch.rand((64, 2048, 3)).cuda(), torch.rand((64, 2048, 3)).cuda()
bs = a.shape[0] # batch_size

results = StructuralLosses.compute_all_metrics(a, b, bs)
print('PointFlow metrics w/o JSD')
print(results)

a_, b_ = a.detach().cpu().numpy(), b.detach().cpu().numpy()

jsd_pcd = StructuralLosses.jsd_between_point_cloud_sets(a_, b_)
print('PointFlow JSD:', jsd_pcd)

PointFlow metrics w/o JSD
{'lgan_mmd-CD': tensor(0.0044, device='cuda:0'), 'lgan_cov-CD': tensor(0.4688, device='cuda:0'), 'lgan_mmd_smp-CD': tensor(0.0044, device='cuda:0'), 'lgan_mmd-EMD': tensor(0.0773, device='cuda:0'), 'lgan_cov-EMD': tensor(0.5312, device='cuda:0'), 'lgan_mmd_smp-EMD': tensor(0.0774, device='cuda:0'), '1-NN-CD-acc_t': tensor(0.6406, device='cuda:0'), '1-NN-CD-acc_f': tensor(0.5156, device='cuda:0'), '1-NN-CD-acc': tensor(0.5781, device='cuda:0'), '1-NN-EMD-acc_t': tensor(0.5156, device='cuda:0'), '1-NN-EMD-acc_f': tensor(0.5781, device='cuda:0'), '1-NN-EMD-acc': tensor(0.5469, device='cuda:0')}
PointFlow JSD: 0.003672857800732565


# MSN-Point-Cloud-Completion


In [4]:
from pathlib import Path
cwd = Path.cwd()
module_dir = cwd / 'pytorch-emd'

# pip uninstall emd
!cd {module_dir} && pip install .

Processing /mnt/c/Users/chawa/OneDrive/CMU_PYTHON/pytorch_pcd_metrics/pytorch-emd
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'emd' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'emd'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for emd: filename=emd-0.0.0-cp39-cp39-linux_x86_64.whl size=149501 sha256=401a8f9bcb87a1d459ba481f51687842b275ef84f4a26e98f57135e00edf3f3c
  Stored in directory: /home/b/.cache/pip/wheels/32/7e/dd/a0af4ba4d6e54a844da619d95f8129c36d87cc2e2112394dbe
Successfully built emd
  Attempting uninstall: emd
    Found existing installation: emd 0.0.0
    Uninstalling emd-0.0.0:
      Successfully uninstalled emd-0.0.0


In [5]:
from emd_wrapper import emdModule, test_emd
test_emd()

Input_size:  torch.Size([20, 8192, 3])
Runtime: 1.882384s
EMD: 0.038827
|set(assignment)|: 8192
Verified EMD: 0.038827


In [6]:
a, b = torch.rand((64, 2048, 3)).cuda(), torch.rand((64, 2048, 3)).cuda()
emd_module = emdModule()
eps, iter = 0.002, 10000
dist, assignment = emd_module(a, b, eps, iter)
print(dist.mean(dim=1), assignment[:5])

tensor([0.0049, 0.0055, 0.0051, 0.0049, 0.0047, 0.0046, 0.0053, 0.0046, 0.0057,
        0.0052, 0.0052, 0.0053, 0.0051, 0.0045, 0.0049, 0.0046, 0.0058, 0.0046,
        0.0047, 0.0048, 0.0052, 0.0055, 0.0047, 0.0051, 0.0050, 0.0048, 0.0053,
        0.0051, 0.0050, 0.0051, 0.0046, 0.0055, 0.0049, 0.0050, 0.0050, 0.0058,
        0.0053, 0.0053, 0.0049, 0.0049, 0.0050, 0.0048, 0.0045, 0.0051, 0.0049,
        0.0052, 0.0048, 0.0046, 0.0049, 0.0052, 0.0053, 0.0051, 0.0051, 0.0052,
        0.0048, 0.0056, 0.0055, 0.0047, 0.0046, 0.0051, 0.0048, 0.0053, 0.0053,
        0.0048], device='cuda:0') tensor([[ 624, 1873, 1864,  ...,  629, 1233,   18],
        [1824,  211, 1249,  ...,  213,  874, 1732],
        [1391, 1473,   60,  ...,  927, 1404, 1233],
        [1549, 1016,    5,  ...,  349, 1502,  732],
        [ 206,  266, 1582,  ...,  573,  254,  402]], device='cuda:0',
       dtype=torch.int32)
